In [1]:
import numpy as np
import scipy

from matplotlib import pyplot as plt
from matplotlib import patches as patches
from matplotlib import cm

import copy

In [2]:
np.random.seed(0)

dtype = np.float32

In [3]:
EPSILON = .025                              # timesteps (seconds)
M       = 1e-21                             # mass (kg)
T_R     = 4e5                               # temperature of random agent (Kelvins)
T_C     = 2e6                               # temperature of causal agent (Kelvins)
K_B     = scipy.constants.Boltzmann         # Boltzmann constant (J/K, or m^2 kg / s^2 K)
TAU     = 10.                               # simulation time horizon

L       = 400                               # length (meters) 
Q_MIN   = np.array([0., 0.  ], dtype=dtype) # minimum box displacement
Q_MAX   = np.array([L , L/5.], dtype=dtype) # maximum box displacement

In [4]:
PREFACTOR = 4.

In [5]:
# T_R *= PREFACTOR
# T_C *= PREFACTOR

In [6]:
timesteps = int(TAU / EPSILON)

In [7]:
def generate_random_fs(num_samples):
    # units of scale are
    # sqrt(kg * (J / K) * K) / s
    # = sqrt(kg * J) / s
    # = sqrt(kg * (kg * m^2 / s^2)) / s
    # = sqrt(kg^2 * m^2 / s^2) / s
    # = (kg * m / s) / s
    # = (kg * m / s^2)
    # = N
    return np.random.normal( loc   = 0. ,
                             scale = 2 * np.sqrt(M * K_B * T_R / EPSILON) ,
                             size  = (num_samples, 2) ).astype(dtype)

In [8]:
def calculate_momentum(p, force):
    p     = force * EPSILON
    p_max = M * np.abs(Q_MAX - Q_MIN) / EPSILON
    return np.sign(p) * np.minimum(p_max, np.abs(p))

In [9]:
def calculate_position(q, p):
    return q  +  EPSILON * p / M

In [10]:
def enforce_elastic_collisions(p, q):
    q  = np.maximum(q, 2 * Q_MIN - q)
    p *= np.sign(q - Q_MIN)
    q  = np.minimum(q, 2 * Q_MAX - q)
    p *= np.sign(Q_MAX - q)
    return p, q

In [11]:
def update_phase_space(p, q, f):
    p    = calculate_momentum(p, f)
    q    = calculate_position(q, p)
    p, q = enforce_elastic_collisions(p, q)
    return p, q

In [12]:
def rollouts(cur_macrostate_p, cur_macrostate_q, num_samples):
    ps = np.tile(cur_macrostate_p[None, :], (num_samples, 1))
    qs = np.tile(cur_macrostate_q[None, :], (num_samples, 1))
    
    paths = copy.deepcopy(qs[None, :])

    fs       = generate_random_fs(num_samples)
    first_fs = copy.deepcopy(fs)
    
    # create MC rollouts producing array (timesteps, num_samples, q_dim)
    for i in range(timesteps):
        ps, qs = update_phase_space(ps, qs, fs)
        paths  = np.append(paths, qs[None, :], axis=0)
        fs     = generate_random_fs(num_samples)

    return paths, first_fs

In [13]:
def log_volume_fracs(paths):
    volume       = 1. / scipy.stats.gaussian_kde(paths).pdf(paths)
    volume_fracs = volume / volume.sum() 
    return np.log(volume_fracs)

In [14]:
def calculate_entropic_force( cur_macrostate_p,
                              cur_macrostate_q ,
                              tau = 10. ,
                              num_samples = 100_000 ,
                              num_subsamples = 10 ):
    
    paths, fs = rollouts(cur_macrostate_p, cur_macrostate_q, num_samples)
    
    # remove the first state because we calculate conditional density
    paths = paths[1:]

    # subsample paths: idea and code from Google Gemini to speed up KDE
    sub_indices = np.linspace( 0 ,
                               timesteps - 1 ,
                               num_subsamples ,
                               dtype=int ) 
    sub_paths   = paths[sub_indices, :, :]
    sub_paths   = sub_paths.transpose((2, 0, 1)).reshape(-1, num_samples)
    
    # calculate entropic force
    return np.mean( fs * log_volume_fracs(sub_paths)[:, None] , axis=0 )

In [15]:
q = np.array([L/10., L/10.])
# q = np.array([L/2., L/10.])
p = np.array([0.   , 0.   ])

path = copy.deepcopy(q[None, :])

for timestep in range(200):
    print(timestep, q)
    f_c  = 2 * T_C / T_R * calculate_entropic_force( p ,
                                                     q ,
                                                     num_samples = 50_000 ,
                                                     num_subsamples = 2 )
    p, q = update_phase_space(p, q, f_c)
    path = np.append(path, q[None, :], axis=0)

0 [40. 40.]
1 [40.03062962 39.64892684]
2 [39.97037418 39.41871217]
3 [40.58668875 39.78776588]
4 [40.34250807 39.70456006]
5 [40.22450521 40.10421236]
6 [39.72511079 40.0808582 ]
7 [39.28544165 40.09699237]
8 [39.79464506 40.44614299]
9 [39.67572807 40.29329453]
10 [39.76089433 40.08149512]
11 [39.8079765  40.25593799]


KeyboardInterrupt: 

In [ ]:
fig, ax = plt.subplots(figsize=(13, 13))
# from https://stackoverflow.com/a/37437395
# ax.add_patch(patches.Rectangle(Q_MIN, L, L/5., edgecolor='k', fill=False))
ax.plot(*path.T)
ax.set_xlim(Q_MIN[0], Q_MAX[0])
ax.set_ylim(Q_MIN[1], Q_MAX[1])
ax.set(aspect='equal')
plt.show()